# Task to experiment
1. Fetch data with schema overwriting
2. Do some basic analysis
3. Best time to use `polars`
4. Compare with `pandas` in respect of both time and efficiency

----------------------------------
- *Calculate time for each task*
----------------------------------

In [34]:
import time
import polars as pl

# 1. Read data as dataframe

- Get csv data (~103 MB)
- Overwrite schema to make sure proper datatype
- `ignore` infer_schema: it scans all rows to find the proper datatype. That's why it slow and risky for large dataset 

- polars code

In [35]:
start_time = time.time()

In [36]:
df = pl.read_csv(source='data8277.csv'
                 , has_header=True
                 , separator=','
                 , try_parse_dates=True
                 , schema_overrides={"count": pl.Int32}
                #  , infer_schema=True  # costly: traberse all rows to find out correct data type
                 , ignore_errors=True
                 , encoding='utf8')

In [37]:
execution_time = time.time() - start_time
print(f"Time to fetch the csv file: {execution_time}")

Time to fetch the csv file: 2.1924350261688232


- pandas code: `wip...`

- findings
    - super fast to parse data 
    - supports polars native data type, not external numpy based datatype
    - try `parse dates param` makes it very efficient to detect datetime related col
    - ignore errors param helps to prevent to break the code while retrieving data

In [38]:
df.head()

Year,Age,Ethnic,Sex,Area,count
i64,i64,i64,i64,i64,i32
2018,0,1,1,1,795
2018,0,1,1,2,5067
2018,0,1,1,3,2229
2018,0,1,1,4,1356
2018,0,1,1,5,180


In [39]:
df.glimpse()  # a snapshot of data

Rows: 34959672
Columns: 6
$ Year   <i64> 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018
$ Age    <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ Ethnic <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ Sex    <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ Area   <i64> 1, 2, 3, 4, 5, 6, 7, 8, 9, 12
$ count  <i32> 795, 5067, 2229, 1356, 180, 738, 630, 1188, 2157, 177



# 2. Basic data analysis

a. get specific columns

- select only cols
- basic calculations with selected cols

b. create dereived col

- to make a derive col from str input as condition use `lit`

c. filter

- basic filtereing
- range

d. sort

e. group by

f. combining DF

## 2.a: Selecting cols

- polars code

In [40]:
start_time_selecting_cols = time.time()

In [41]:
df_year_age = df.select(['Year', 'Age'])
df_year_age.head()

Year,Age
i64,i64
2018,0
2018,0
2018,0
2018,0
2018,0


In [42]:
df_year_age = df.select(
    pl.col('Year')
    , (pl.col('Age') * 1.0).alias('Age*1.0')
)
df_year_age.head()

Year,Age*1.0
i64,f64
2018,0.0
2018,0.0
2018,0.0
2018,0.0
2018,0.0


In [43]:
execution_time_for_selecting_cols = time.time() - start_time_selecting_cols
print(f"execution time for selecting cols: {execution_time_for_selecting_cols}")

execution time for selecting cols: 0.17614006996154785


- pandas code: `wip...`

## 2.b: Derive col

- polars code

In [44]:
start_time_creating_derive_cols = time.time()

In [45]:
df_derive = df.with_columns(
    gender = pl.when(pl.col("Sex") == 1)
    .then(pl.lit('male'))
    .when(pl.col("Sex") == 2)
    .then(pl.lit('female'))
    .otherwise(pl.lit('others'))
    )

df_derive = df_derive.drop(['Sex'])
df_derive.head()

Year,Age,Ethnic,Area,count,gender
i64,i64,i64,i64,i32,str
2018,0,1,1,795,"""male"""
2018,0,1,2,5067,"""male"""
2018,0,1,3,2229,"""male"""
2018,0,1,4,1356,"""male"""
2018,0,1,5,180,"""male"""


In [46]:
execution_time_creating_derive_cols = time.time() - start_time_creating_derive_cols
print(f"time to create derive col: {execution_time_creating_derive_cols}")

time to create derive col: 0.28652071952819824


- pandas code: `wip...`

## 2.c: Filter

- polars code

In [47]:
start_time_for_filtering = time.time()

In [48]:
df_basic_filter = df_derive.filter(
        df_derive['Year'] < 2007
    )

df_basic_filter.head()

Year,Age,Ethnic,Area,count,gender
i64,i64,i64,i64,i32,str
2006,0,1,1,615,"""male"""
2006,0,1,2,5142,"""male"""
2006,0,1,3,1809,"""male"""
2006,0,1,4,1110,"""male"""
2006,0,1,5,168,"""male"""


In [49]:
df_basic_filter_range = df_derive.filter(
    df_derive['Year'].is_between(2006, 2013)  # upper limit inclusive
    )

df_basic_filter_range.head()

Year,Age,Ethnic,Area,count,gender
i64,i64,i64,i64,i32,str
2013,0,1,1,660,"""male"""
2013,0,1,2,5502,"""male"""
2013,0,1,3,1971,"""male"""
2013,0,1,4,1212,"""male"""
2013,0,1,5,168,"""male"""


In [50]:
execution_time_for_filtering = time.time() - start_time_for_filtering
print(f"time to filter data: {execution_time_for_filtering}")

time to filter data: 0.13495707511901855


- pandas code: `wip...`

## 2.d: Sort

- polars code

In [51]:
start_time_for_sorting = time.time()

In [52]:
df_derive = df_derive.sort(by=["Year", "count"],
                            nulls_last=True)
df_derive.head()

Year,Age,Ethnic,Area,count,gender
i64,i64,i64,i64,i32,str
2006,0,5,5,0,"""male"""
2006,0,5,12,0,"""male"""
2006,0,5,15,0,"""male"""
2006,0,5,16,0,"""male"""
2006,0,5,1,0,"""female"""


In [53]:
execution_time_for_sorting = time.time() - start_time_for_sorting
print(f"Time to sort data: {execution_time_for_sorting}")

Time to sort data: 3.9916300773620605


- pandas code: `wip...`

## 2.e: Group by

In [54]:
start_time_for_grouping = time.time()

In [55]:
df_year_wise_count = df_derive.group_by(
    ["Year"],
    maintain_order=True
    ).agg(
        pl.col("count"). \
        sum(). \
        alias('year_wise_total_count'),

        pl.col("count"). \
        mean(). \
        round(2). \
        alias('year_wise_avg_count'),

        pl.col('gender')
        )

df_year_wise_count.head()


Year,year_wise_total_count,year_wise_avg_count,gender
i64,i32,f64,list[str]
2006,1049590488,204.45,"[""male"", ""male"", … ""others""]"
2013,1072735711,202.39,"[""male"", ""male"", … ""others""]"
2018,1217611015,212.4,"[""male"", ""male"", … ""others""]"


In [56]:
execution_time_for_grouping = time.time() - start_time_for_grouping
print(f"time to aggregate data: {execution_time_for_grouping}")

time to aggregate data: 0.16802120208740234


- pandas code: `wip...`

## 2.f: combining DF

i. joining. [doc](https://docs.pola.rs/user-guide/transformations/joins/#quick-reference-table)

ii. concat

In [57]:
df2 = pl.DataFrame(
    {
        "Year": [2006, 2013, 2018, 2019],
    }
)

df2.head()

Year
i64
2006
2013
2018
2019


## 2.f.i: joining

- polars code

In [58]:
start_time_for_joining = time.time()

In [59]:
df_left_join = df2.join(df, 
                       on="Year",
                       how="left"). \
                        sort(by=["Year"], 
                             descending=True)

df_left_join.head()

Year,Age,Ethnic,Sex,Area,count
i64,i64,i64,i64,i64,i32
2019,null,null,null,null,null
2018,0,1,1,1,795
2018,0,1,1,2,5067
2018,0,1,1,3,2229
2018,0,1,1,4,1356


In [60]:
df_inner_join = df.join(df2, 
                       on="Year",
                       how="inner").sort(by=["Year"], descending=True)

df_inner_join.head()

Year,Age,Ethnic,Sex,Area,count
i64,i64,i64,i64,i64,i32
2018,0,1,1,1,795
2018,0,1,1,2,5067
2018,0,1,1,3,2229
2018,0,1,1,4,1356
2018,0,1,1,5,180


In [61]:
execution_time_for_joining = time.time() - start_time_for_joining
print(f"Time to join data: {execution_time_for_joining}")

Time to join data: 3.6304609775543213


- pandas code: `wip...`

- findings
    - despite having traditional joins, it has some extra join techniques like `semi`, `anti` like `PySpark`

## 2.f.ii: concatinating

In [62]:
df3 = pl.DataFrame(
    {
        "Year": [2020, 2021, 2022, 2023],
        "Age": [0,0,0,0],
        "Ethnic": [1, 2, 3, 4],
        "Sex": [1, 2, 1, 2], 
        "Area": [1, 2, 3, 4],
        "count": [1000, 2000, 3000, 4000]
    }
)

- polars code

In [63]:
start_time_for_concatinating = time.time()

In [64]:
df_concat = pl.concat([df, df3], 
                      how="vertical_relaxed")  # vertical_relaxed: best for datatype missmatched, Int32 -> Int64

df_concat.sort(by='Year', 
                      descending=True).head(5)

Year,Age,Ethnic,Sex,Area,count
i64,i64,i64,i64,i64,i64
2023,0,4,2,4,4000
2022,0,3,1,3,3000
2021,0,2,2,2,2000
2020,0,1,1,1,1000
2018,0,1,1,1,795


In [65]:
execution_time_for_concatinating = time.time() - start_time_for_concatinating
print(f"time to concat data: {execution_time_for_concatinating}")

time to concat data: 1.5106580257415771


- pandas code: `wip...`